In [6]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import utils as ut

def biggestContours(contours,numberOfContours=1):
    # Sort the contours by area in descending order
    contours = [np.array(contour) for contour in contours]
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:numberOfContours]
    return contours

def computeCenter(contours):
    # Combine contours into one array
    if len(contours)==0:
        return None
    
    combined_contour = np.vstack(contours)

    # Compute bounding box for the combined contours
    x, y, w, h = cv2.boundingRect(combined_contour)

    # Calculate center point of the bounding box
    center_x = x + w // 2
    center_y = y + h // 2

    return (center_x, center_y)

def find_missing_point(point1, point2, known_point):  
    point1 = list(point1.ravel())
    point2 = list(point2.ravel())
    known_point = list(known_point.ravel())
    # Determine the direction vector of the first line
    direction_vector1 = np.array(point2) - np.array(point1)
    
    # Find the equation of the first line (y = mx + c)
    m1 = direction_vector1[1] / direction_vector1[0]  # Slope
    c1 = point1[1] - m1 * point1[0]  # Intercept
    
    
    # Find the equation of the second line
    m2 = m1  # Lines are parallel, so slopes are equal
    c2 = known_point[1] - m2 * known_point[0]  # Intercept

    # Calculate the length of the vector
    vector_length = np.linalg.norm(direction_vector1)
    
    # Find the x-coordinate of the intersection point
    intersection_x = known_point[0] + vector_length*np.cos(np.arctan(m1))
    
    # Find the y-coordinate of the intersection point
    intersection_y = known_point[1] + vector_length*np.sin(np.arctan(m1))
    
    return intersection_x, intersection_y


imagePath='TC/06.png'
image = cv2.imread(imagePath)

contours= ut.locatorContours2(image=image,invertColors=False,showImages=False)

locatorBoxes=biggestContours(contours,numberOfContours=3)

centerPoint=computeCenter(locatorBoxes)
cv2.circle(center:=image.copy(), centerPoint, 10, (0, 255, 0), -1)
ut.showImage(ut.drawContourPoints(image.copy(),locatorBoxes))

ut.showImage(center)

for i,locatorbox in enumerate(locatorBoxes):  # order all points in each box
    locatorBoxes[i]=ut.orderPoints(locatorbox)

topLeft= topRight= bottomRight= bottomLeft = 0  # corner points for the whole qr code
for locator in locatorBoxes:
    x, y, w, h = cv2.boundingRect(locator)

    # Calculate center point of the bounding box
    center_x,center_y = x + w // 2,y + h // 2
    
    if center_x> centerPoint[0] and center_y< centerPoint[1]:
        topRight=locator[1]
    
    # elif center_x> centerPoint[0] and center_y> centerPoint[1]:
        # bottomRight=locator[3]
    
    elif center_x< centerPoint[0] and center_y< centerPoint[1]:
        topLeft=locator[0]
    
    elif center_x< centerPoint[0] and center_y> centerPoint[1]:
        bottomLeft=locator[2]

cv2.circle(center, topRight.ravel(), 10, (0, 0, 255), -1)
cv2.circle(center, bottomLeft.ravel(), 10, (0, 0, 255), -1)
cv2.circle(center, topLeft.ravel(), 10, (0, 0, 255), -1)
# cv2.circle(center, bottomRight.ravel(), 10, (0, 0, 255), -1)
bottomRight=find_missing_point(topLeft,topRight,bottomLeft)
bottomRight=(int(round(bottomRight[0])),int(round(bottomRight[1])))
bottomRight=np.array([bottomRight])
cv2.circle(center, bottomRight.ravel(), 10, (0, 0, 255), -1)

width,height = 1012,1012


pts1 = np.float32([topLeft.ravel(),topRight.ravel(),bottomLeft.ravel(),bottomRight.ravel()])
pts2 = np.float32([[0,0],[0,width],[height,0],[width,height]])



matrix = cv2.getPerspectiveTransform(pts1,pts2)
output = cv2.warpPerspective(image,matrix,(width,height))

# cv2.imwrite('emad.png',output)

ut.showImage(output)


AttributeError: module 'utils' has no attribute 'locatorContours2'